In [90]:
import pandas as pd
import yfinance as yf
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import yfinance as yf
import vectorbt as vbt
from ta.trend import MACD


from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator
from datetime import datetime
from datetime import timedelta as td
from tqdm import tqdm  # Visualize loop progress
from sklearn.linear_model import LinearRegression

pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
vbt.settings.set_theme('dark')
# vbt.settings.plotting.aut_rangebreaks = True

import requests
import json
import os
import datetime
import pytz # timezone

In [91]:
from dotenv import load_dotenv
load_dotenv()

True

| Gap-Up Screener |
|--|
TradingView Data (import)

In [92]:
# today's date string for file naming automation
# today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
# today_date_str = '_2024-06-11.csv' # this is a near perfect outcome example replay day
# today_date_str = '_2024-06-26.csv' # test data sprouts farmers market should be first
today_date_str = '_2024_07-11.csv'

print(today_date_str)

_2024_07-11.csv


In [93]:
# concat base file name with today's date string
# filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
filename = "trading_view_raw_data/tv_screen_gap-up_2024-07-11_0611.csv"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# trading_view_raw_df.head(5)

1782 U.S. Stocks 
$(pre-market change) > $0.00 USD


In [94]:
#### CREATE MARKET CAP CATEGORIES ####
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans - 200 billion and above
        (df['Market capitalization'] >= 200000000000),
        # Large Cap - 10 billion to 200 billion
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),
        # Midlers - 2 billion to 10 billion
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),
        # Small Cap - 300 million to 2 billion
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000), 
        # Micro Cap - 50 million to 300 million
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),
        # Shrimp - 50 million and below
        (df['Market capitalization'] <= 50000000)
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df

In [95]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)
print(len(trading_view_raw_df.index))

1782


In [96]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [97]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1782 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Small caps,461,25.869809
1,Midlers,417,23.400673
2,Large caps,389,21.829405
3,Shrimp,269,15.095398
4,Micro caps,216,12.121212
5,Titans,22,1.234568
6,Undefined,8,0.448934


In [98]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(2)

651 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Relative Volume at Time,Relative Volume 1 day,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 week,Relative Volume 1 month,Highest high 1 month,Highest high 1 month - Currency,Highest high 3 months,Highest high 3 months - Currency,Highest high 6 months,Highest high 6 months - Currency,Highest high 52 weeks,Highest high 52 weeks - Currency,Highest high All Time,Highest high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes,marketCapType
46,HON,Honeywell International Inc.,Aerospace & defense,Electronic technology,NASDAQ,"S&P 500, NASDAQ 100, Dow Jones Industrial Average, NASDAQ Composite, S&P 500 Industrials, S&P 100, Russell 3000, Dow Jones Composite Average, Russell 1000, NASDAQ Industrials",1.396728e+11,USD,214.49,USD,214.50,USD,1.11,USD,0.517507,0.004662,6.505895e+08,1791777.0,5005250.0,696,2039615.2,2.684721e+06,2.745367e+06,2.059165,1.197315,1.375244,213.398333,24.860335,0.982004,0.830487,0.771710,0.953374,1.031849,2.837554,5.149089,7.169905,5.363432,2.609825,1.393444,1.229425,0.393318,0.192267,218.36,USD,218.36,USD,218.36,USD,218.36,USD,236.86,USD,214.83,USD,214.94,USD,214.94,USD,214.94,USD,214.94,USD,214.94,USD,214.94,USD,214.94,USD,215.53,USD,3.857579e+05,USD,214.5120,214.637500,214.4132,214.277677,214.964345,214.388235,213.812125,214.767827,214.083985,213.400143,US4385161066,0.830487,2024-04-25,2024-07-25,Buy,229.803333,USD,Buy,Large caps
39,BX,Blackstone Inc.,Investment managers,Finance,NYSE,"S&P 500, Russell 3000, S&P 500 Financials, Russell 1000",1.502364e+11,USD,124.26,USD,125.24,USD,1.63,USD,1.311766,0.788669,7.072880e+08,1976374.0,6742436.0,5747,2492813.7,2.761144e+06,3.366123e+06,2.547193,2.007773,1.806962,123.313333,43.821413,0.745964,0.796513,1.828267,1.732728,1.490708,5.540136,11.219818,7.185012,4.951053,2.156885,1.011424,0.858701,0.467538,0.182838,126.55,USD,131.34,USD,133.56,USD,133.56,USD,149.78,USD,124.37,USD,124.37,USD,124.37,USD,124.37,USD,124.37,USD,124.37,USD,124.37,USD,124.37,USD,125.58,USD,2.169991e+06,USD,124.2678,124.215775,123.9192,123.658600,124.406846,123.960610,123.514374,124.058302,123.557295,123.056288,US09260D1072,0.796513,2024-04-18,2024-07-18,Buy,124.722222,USD,Buy,Large caps


In [99]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

print(tv_df.shape)

(651, 91)


In [100]:
market_info_df = tv_df['marketCapType'].value_counts().reset_index()
market_info_df.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_info_df['percentage'] = (market_info_df['count'] / market_info_df['count'].sum()) * 100

print(market_info_df.shape)
market_info_df

(5, 3)


,marketCapType,count,percentage
0,Large caps,248,38.095238
1,Midlers,234,35.944700
2,Small caps,138,21.198157
3,Titans,18,2.764977
4,Micro caps,13,1.996928


In [101]:
# need to add error handling for when zero (0) return (4/4/24)
# zero again 5/24/24
# - not so much error handling but want to see if we need to adjust the criteria in general
# - need to put this theory and algo through more paper trading and adjust as needed
# think about adding other staples to dataframe
# or you create a separate investments_long_df or similar for those, i.e., TSLA, PLTR, SOFI etc...

| FILTER CRITERIA CONFIG |
|-|
| TradingView (raw data) | 

things to add;
* Pre-Market Volume: Adding a minimum pre-market volume threshold can help filter out stocks that might have large percentage changes but low trading volumes, which can lead to unreliable price movements and high slippage.
Example: A minimum of 50,000 shares traded in the pre-market for all categories.

* overall market sentiment and filter tightener (macro)

In [102]:
#### CRITERIA CONFIG DICTIONARY ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # 1 billion shares
        "relative_volume_threshold": 1.2,  # More inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Mid caps": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

In [103]:
# #### THIS IS THE TEST DICTIONARY - COMPARE RESULTS ####
# #### CRITERIA CONFIG DICTIONARY ####
# criteria_config = {
#     "Titans": {
#         "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
#         "float_shares_outstanding_threshold": 1000000000,  # 1 billion shares
#         "relative_volume_threshold": 1.2,  # More inclusive
#         "relative_volume_at_time_threshold": 0.03,  # More inclusive
#         "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
#         "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Large caps": {
#         "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
#         "float_shares_outstanding_threshold": 200000000,  # 200 million shares
#         "relative_volume_threshold": 1.3,  # More inclusive
#         "relative_volume_at_time_threshold": 0.04,  # More inclusive
#         "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
#         "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Mid caps": {
#         "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
#         "float_shares_outstanding_threshold": 50000000,  # 50 million shares
#         "relative_volume_threshold": 1.3,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.02,
#         "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Small caps": {
#         "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
#         "float_shares_outstanding_threshold": 20000000,  # 20 million shares
#         "relative_volume_threshold": 1.2,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.03,
#         "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Micro caps": {
#         "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
#         "float_shares_outstanding_threshold": 5000000,  # 5 million shares
#         "relative_volume_threshold": 1.1,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.04,
#         "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Shrimp": {
#         "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
#         "float_shares_outstanding_threshold": 1000000,  # 1 million shares
#         "relative_volume_threshold": 1.0,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.05,
#         "pre_market_vwap_drawdown_threshold": 0.008,  # 0.8% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     }
# }


In [104]:
# #### CRITERIA CONFIG DICTIONARY ####
# criteria_config = {
#     "Titans": {
#         "pre_market_change_pct_threshold": 0.003,  # 0.3% for Titans
#         "float_shares_outstanding_threshold": 1000000000,  # 1 billion shares
#         "relative_volume_threshold": 1.3,  # More inclusive
#         "relative_volume_at_time_threshold": 0.03,  # More inclusive
#         "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
#         "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Large caps": {
#         "pre_market_change_pct_threshold": 0.007,  # 0.7% for Large caps
#         "float_shares_outstanding_threshold": 200000000,  # 200 million shares
#         "relative_volume_threshold": 1.4,  # More inclusive
#         "relative_volume_at_time_threshold": 0.04,  # More inclusive
#         "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
#         "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Mid caps": {
#         "pre_market_change_pct_threshold": 0.025,  # 2.5% for Mid caps
#         "float_shares_outstanding_threshold": 50000000,  # 50 million shares
#         "relative_volume_threshold": 1.4,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.02,
#         "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Small caps": {
#         "pre_market_change_pct_threshold": 0.035,  # 3.5% for Small caps
#         "float_shares_outstanding_threshold": 20000000,  # 20 million shares
#         "relative_volume_threshold": 1.3,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.03,
#         "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Micro caps": {
#         "pre_market_change_pct_threshold": 0.045,  # 4.5% for Micro caps
#         "float_shares_outstanding_threshold": 5000000,  # 5 million shares
#         "relative_volume_threshold": 1.2,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.04,
#         "pre_market_vwap_drawdown_threshold": 0.008,  # 0.8% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     },
#     "Shrimp": {
#         "pre_market_change_pct_threshold": 0.055,  # 5.5% for Shrimp
#         "float_shares_outstanding_threshold": 1000000,  # 1 million shares
#         "relative_volume_threshold": 1.1,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.05,
#         "pre_market_vwap_drawdown_threshold": 0.009,  # 0.9% drawdown from VWAP
#         "pre_market_volume_threshold": 75000  # Minimum pre-market volume
#     }
# }


In [105]:
# what about exceptions, like if all these criteria above are met but there is super high volume and less than x& < 1% gap-up
# would have to specify specific criteria for the market caps

In [106]:
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    df.loc[:, 'Float shares outstanding'] = pd.to_numeric(df['Float shares outstanding'], errors='coerce')
    return df[df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df.loc[:, 'Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    df.loc[:, 'Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vwap_drawdown_threshold):
    """Filter stocks where the price is near the VWAP, considering the specified drawdown threshold."""
    df.loc[:, 'Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df.loc[:, 'Volume Weighted Average Price 1 day'] = pd.to_numeric(df['Volume Weighted Average Price 1 day'], errors='coerce')
    df.loc[:, 'Min Price from VWAP'] = df['Volume Weighted Average Price 1 day'] * (1 - pre_market_vwap_drawdown_threshold)
    return df[df['Price'] >= df['Min Price from VWAP']].drop(columns=['Min Price from VWAP'])

def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    df.loc[:, 'Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df.loc[:, 'Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df.loc[:, 'Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    return df[(df['Volatility 1 day'] >= df['Volatility 1 week']) & (df['Volatility 1 day'] >= df['Volatility 1 month'])]

#### last (i.e., stay organized, bookend filter section here, add other filters above ####
def filter_by_pre_market_volume(df, pre_market_volume_threshold):
    """Filter stocks by pre-market volume."""
    df['Pre-market Volume'] = pd.to_numeric(df['Pre-market Volume'], errors='coerce')
    return df[df['Pre-market Volume'] >= pre_market_volume_threshold]

In [107]:
# WRAPPER FUNCTION
def screen_stocks_by_category(df, category):
    # FILTERS
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vwap_drawdown_threshold = config.get("pre_market_vwap_drawdown_threshold", 0)
    # last
    pre_market_volume_threshold = config.get("pre_market_volume_threshold", 0)

    # FUNCTIONS (core processing)
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df)
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vwap_drawdown_threshold)
    # last
    filtered_df = filter_by_pre_market_volume(filtered_df, pre_market_volume_threshold)

    return filtered_df

In [108]:
#### EXECUTE AND CREATE FULL (baseline) GAP-UP DATAFRAME ####
gap_up_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    gap_up_df = pd.concat([gap_up_df, gap_up_stage_df])

# filter in descending order by market cap and then by pre-market change percentage
gap_up_df = gap_up_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
gap_up_df.reset_index(drop=True, inplace=True)

# # display results
# print(f'Original dataFrame length: {len(tv_df.index)}')
# print(f'Number of rows removed: {len(tv_df.index) - len(gap_up_df.index)}')
# filter in descending order by market cap and then by pre-market change percentage
gap_up_df = gap_up_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# print(f'NEW (Gap-Up) dataFrame length: {len(gap_up_df.index)} (target stocks)')

# reset index and drop
gap_up_df.reset_index(drop=True, inplace=True)

## LOOK AT COLUMNS WHEN NEEDED
# print(gap_up_df.columns)
print(gap_up_df.shape)
gap_up_df.head(2)

(64, 91)


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Relative Volume at Time,Relative Volume 1 day,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 week,Relative Volume 1 month,Highest high 1 month,Highest high 1 month - Currency,Highest high 3 months,Highest high 3 months - Currency,Highest high 6 months,Highest high 6 months - Currency,Highest high 52 weeks,Highest high 52 weeks - Currency,Highest high All Time,Highest high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes,marketCapType
0,WIX,Wix.com Ltd.,Information technology services,Technology services,NASDAQ,"NASDAQ Composite, NASDAQ Computer",9.070821e+09,USD,163.01,USD,163.01,USD,0.88,USD,0.539844,0.000000,5.345942e+07,681487.0,1608469.0,69777,465526.7,6.061127e+05,6.254867e+05,4.122332,3.027283,2.269606,161.073333,145.752861,1.663193,1.597054,1.696484,1.211676,1.470896,2.246410,7.114543,7.968133,5.183866,2.288230,1.468125,1.15192,0.431155,0.273122,172.63,USD,178.65,USD,178.65,USD,178.65,USD,362.07,USD,163.10,USD,163.19,USD,163.19,USD,163.19,USD,163.19,USD,163.19,USD,163.26,USD,163.26,USD,163.26,USD,2.945426e+05,USD,162.814,162.841875,162.730,162.339808,163.161513,162.72635,162.291187,163.084750,162.05900,161.033250,IL0011301780,1.597054,2024-05-20,2024-08-07,Buy,188.222222,USD,Strong buy,Midlers
1,PNW,Pinnacle West Capital Corporation,Electric utilities,Utilities,NYSE,"S&P 500, S&P 500 Utilities, Russell 3000, Russell 1000, PHLX Utilities Sector",8.913237e+09,USD,78.49,USD,78.85,USD,0.37,USD,0.471398,0.458657,1.133293e+08,1183249.0,3210117.0,230,1031013.5,1.116733e+06,1.057154e+06,1.906615,1.673933,1.603082,78.053333,17.124468,1.512568,1.209774,-0.021277,0.571245,0.504270,5.656846,5.150998,5.854853,6.612497,2.873743,1.745755,1.80288,0.700387,0.290820,78.57,USD,78.89,USD,78.89,USD,86.03,USD,105.51,USD,78.57,USD,78.57,USD,78.57,USD,78.57,USD,78.57,USD,78.57,USD,78.57,USD,78.57,USD,78.57,USD,5.797520e+07,USD,78.467,78.415000,78.326,78.289231,78.510370,78.34525,78.180130,78.573093,78.17275,77.772407,US7234841010,1.209774,2024-05-02,2024-08-01,Buy,78.750000,USD,Strong buy,Midlers


| SCREEN LEVEL I |
|-|
| First TARGETs | 

In [109]:
#### Gap-Up (potential, shortlisted) TARGETs ####
# might want to add pre-market change $, pre-market change %
print(f'{len(gap_up_df.index)} Stocks')
gap_up_df.iloc[:, [0, 1, 8]].head(20)

64 Stocks


,Symbol,Description,Price
0,WIX,Wix.com Ltd.,163.01
1,PNW,Pinnacle West Capital Corporation,78.49
2,FRT,Federal Realty Investment Trust,102.61
3,CGNX,Cognex Corporation,49.34
4,ROIV,Roivant Sciences Ltd.,11.25
5,RBC,RBC Bearings Incorporated,282.27
6,SFM,"Sprouts Farmers Market, Inc.",80.44
7,NNN,"NNN REIT, Inc.",42.76
8,FLR,Fluor Corporation,44.77
9,WBS,Webster Financial Corporation,44.47


In [110]:
#### CREATE SIMPLE GAP-UP DATAFRAME ####
simple_cols_list = ['Symbol', 'Description', 'Pre-market Change %', 'Pre-market Gap %', 'marketCapType', 'Price', 'Pre-market Open', 'Industry', 'Index', 'Sector', 'Exchange']
simple_gap_df = gap_up_df[simple_cols_list]

# print(simple_gap_df.columns)
first_list = simple_gap_df['Symbol'].unique().tolist()

print(f'{len(simple_gap_df)} (target stocks) TODAY') # fix this to dynamic
print(first_list)
simple_gap_df.head(2)

64 (target stocks) TODAY
['WIX', 'PNW', 'FRT', 'CGNX', 'ROIV', 'RBC', 'SFM', 'NNN', 'FLR', 'WBS', 'ENSG', 'CBSH', 'EXP', 'OGE', 'RMBS', 'CMA', 'AGI', 'ZION', 'BPOP', 'SUM', 'JAZZ', 'SNV', 'TMHC', 'VVV', 'WHR', 'MOD', 'SITE', 'COOP', 'R', 'ATKR', 'ZWS', 'FNB', 'KBH', 'AVAV', 'COLB', 'GBCI', 'PAGS', 'FFIN', 'UMBF', 'HGV', 'URBN', 'IPGP', 'ALE', 'SHAK', 'VLY', 'APLE', 'AX', 'QFIN', 'AROC', 'APAM', 'YETI', 'GFF', 'FULT', 'NHI', 'SKT', 'MWA', 'ABR', 'LXP', 'FHB', 'CBU', 'EBC', 'WAFD', 'BFH', 'PPBI']


,Symbol,Description,Pre-market Change %,Pre-market Gap %,marketCapType,Price,Pre-market Open,Industry,Index,Sector,Exchange
0,WIX,Wix.com Ltd.,0.539844,0.000000,Midlers,163.01,163.01,Information technology services,"NASDAQ Composite, NASDAQ Computer",Technology services,NASDAQ
1,PNW,Pinnacle West Capital Corporation,0.471398,0.458657,Midlers,78.49,78.85,Electric utilities,"S&P 500, S&P 500 Utilities, Russell 3000, Russell 1000, PHLX Utilities Sector",Utilities,NYSE


In [111]:
# def compute_technical_indicators(df):
#     tech_data = []

#     for index, row in df.iterrows():
#         symbol = row['Symbol']
#         data = yf.download(symbol, period="6mo", interval="1d")

#         if not data.empty:
#             ma20 = SMAIndicator(data['Close'], window=20).sma_indicator().iloc[-1]
#             ma50 = SMAIndicator(data['Close'], window=50).sma_indicator().iloc[-1]
#             rsi = RSIIndicator(data['Close'], window=14).rsi().iloc[-1]

#             tech_data.append([symbol, row['Current Price'], row['Timestamp'], ma20, ma50, rsi])

#     tech_df = pd.DataFrame(tech_data, columns=['Symbol', 'Current Price', 'Timestamp', 'MA20', 'MA50', 'RSI'])
#     return tech_df

# def filter_with_technical_indicators(df):
#     filtered_stocks = []

#     for index, row in df.iterrows():
#         if row['Current Price'] > row['MA20'] and row['Current Price'] > row['MA50'] and 30 < row['RSI'] < 70:
#             filtered_stocks.append(row)

#     filtered_df = pd.DataFrame(filtered_stocks)
#     return filtered_df

In [112]:
# technical_indicators_df = compute_technical_indicators(premarket_prices_df)
# final_filtered_df = filter_with_technical_indicators(technical_indicators_df)

In [113]:
# # save to xlsx
# simple_gap_df.to_excel('gap_up_stocks_4.xlsx', index=False)

| SCREEN LEVEL II |
|-|
| Secondary TARGETs |

In [114]:
def compute_technical_indicators(df):
    tech_data = []

    for symbol in df['Symbol'].unique():
        data = yf.download(symbol, period="6mo", interval="1d")

        if not data.empty:
            rsi = RSIIndicator(data['Close'], window=14).rsi().iloc[-1]
            ma20 = SMAIndicator(data['Close'], window=20).sma_indicator().iloc[-1]
            ma50 = SMAIndicator(data['Close'], window=50).sma_indicator().iloc[-1]
            macd = MACD(data['Close']).macd_diff().iloc[-1]

            tech_data.append({
                'Symbol': symbol,
                'RSI': rsi,
                'MA20': ma20,
                'MA50': ma50,
                'MACD': macd
            })

    tech_df = pd.DataFrame(tech_data)
    return tech_df

In [115]:
# execute -> compute technical indicators
technicals_df = compute_technical_indicators(simple_gap_df)
# execute -> merge the technical indicators back to the simple gap dataframe
simple_gap_df = simple_gap_df.merge(technicals_df, on='Symbol')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [116]:
def screen_stocks_by_technical_indicators(df):
    # Filter based on RSI, MA, and MACD
    df = df[(df['RSI'] > 30) & (df['RSI'] < 70)]
    df = df[df['Price'] > df['MA20']]
    df = df[df['Price'] > df['MA50']]
    df = df[df['MACD'] > 0]

    return df

# Apply the technical indicator filtering
enhanced_gap_df = screen_stocks_by_technical_indicators(simple_gap_df)

print(f"Level I -> {simple_gap_df.shape[0]} (stocks)")
print(f"Level II -> {enhanced_gap_df.shape[0]} (stocks)\n")

print(enhanced_gap_df.columns)
# print(enhanced_gap_df.columns)
# enhanced_gap_df.head(2)

Level I -> 64 (stocks)
Level II -> 24 (stocks)

Index(['Symbol', 'Description', 'Pre-market Change %', 'Pre-market Gap %',
       'marketCapType', 'Price', 'Pre-market Open', 'Industry', 'Index',
       'Sector', 'Exchange', 'RSI', 'MA20', 'MA50', 'MACD'],
      dtype='object')


In [117]:
# return a list of symbols from the simple_gap_df
suspect_list = enhanced_gap_df['Symbol'].unique().tolist()

print(suspect_list)
print(len(suspect_list))

['PNW', 'FRT', 'ROIV', 'RBC', 'NNN', 'FLR', 'WBS', 'CBSH', 'RMBS', 'BPOP', 'SNV', 'VVV', 'WHR', 'MOD', 'FNB', 'GBCI', 'UMBF', 'URBN', 'IPGP', 'ALE', 'QFIN', 'AROC', 'FULT', 'MWA']
24


In [118]:
def get_premarket_prices(symbols):
    premarket_data = []

    for symbol in symbols:
        # Retrieve the historical data for the last two days including pre/post market data
        data = yf.download(symbol, period="2d", interval="1m", prepost=True)
        
        if not data.empty:
            # Filter pre-market data
            premarket_data_today = data.between_time('04:00', '09:30')
            
            if not premarket_data_today.empty:
                # Get the last pre-market price and timestamp
                current_price = premarket_data_today.iloc[-1]['Close']
                timestamp = premarket_data_today.index[-1]
                
                premarket_data.append([symbol, current_price, timestamp])

    # Create a DataFrame with columns Symbol, Current Price, and Timestamp
    premarket_prices_df = pd.DataFrame(premarket_data, columns=['Symbol', 'Current Price', 'Timestamp'])

    return premarket_prices_df

In [139]:
# get the pre-market prices for the symbols
premarket_df = get_premarket_prices(suspect_list)

print(premarket_df.shape)
print(premarket_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

(24, 3)
   Symbol  Current Price Timestamp                
0    PNW    78.940002    2024-07-11 09:30:00-04:00
1    FRT   104.120003    2024-07-11 09:30:00-04:00
2   ROIV    11.410100    2024-07-11 09:30:00-04:00
3    RBC   285.010010    2024-07-11 09:30:00-04:00
4    NNN    43.270000    2024-07-11 09:30:00-04:00
5    FLR    45.619999    2024-07-11 09:30:00-04:00
6    WBS    45.150002    2024-07-11 09:30:00-04:00
7   CBSH    57.660000    2024-07-11 09:30:00-04:00
8   RMBS    67.419998    2024-07-11 09:30:00-04:00
9   BPOP    91.889999    2024-07-11 09:30:00-04:00
10   SNV    41.200001    2024-07-11 09:30:00-04:00
11   VVV    45.205002    2024-07-11 09:30:00-04:00
12   WHR   105.099998    2024-07-11 09:30:00-04:00
13   MOD   107.000000    2024-07-11 09:30:00-04:00
14   FNB    14.140000    2024-07-11 09:30:00-04:00
15  GBCI    38.529999    2024-07-11 09:30:00-04:00
16  UMBF    87.349998    2024-07-11 09:30:00-04:00
17  URBN    45.139999    2024-07-11 09:30:00-04:00
18  IPGP    87.750000  

In [ ]:
# create two cols from time stamp, date and yahoo_finance_timestamp. date format '2024-07-11', and yahoo_finance_timestamp format '04:00:00'
# keep original timestamp for reference
# change current price to pre_market_price@time

# # create new columns
# premarket_df['Date'] = premarket_df['Timestamp'].dt.strftime('%Y-%m-%d')
# premarket_df['yahoo_finance_timestamp'] = premarket_df['Timestamp'].dt.strftime('%H:%M:%S')
# premarket_df['price_yahoo_kinda_now'] = premarket_df['Current Price']
# # drop original timestamp
# premarket_price_now_df.drop(columns=['Timestamp'], inplace=True)

# print(premarket_df.shape)

# premarket_df

In [141]:
#### ---- Quick Stary Backtesting ---- ####
def add_stock_data(df):
    symbols = df['Symbol'].tolist()
    stock_data = {}

    for symbol in symbols:
        stock_info = yf.Ticker(symbol)
        today_data = stock_info.history(period='1d')
        
        if not today_data.empty:
            stock_data[symbol] = {
                'High': today_data['High'].values[0],
                'Low': today_data['Low'].values[0],
                'Open': today_data['Open'].values[0],
                'Close': today_data['Close'].values[0],
                'Date': today_data.index[0]
            }
        else:
            stock_data[symbol] = {
                'High': None,
                'Low': None,
                'Open': None,
                'Close': None,
                'Date': None
            }

    # create a DataFrame from the stock data dictionary
    stock_data_df = pd.DataFrame(stock_data).T.reset_index().rename(columns={'index': 'Symbol'})
    
    # merge the new stock data with the existing DataFrame
    updated_df = pd.merge(df, stock_data_df, on='Symbol', how='left')
    
    return updated_df

# fetch prices at specific times
def add_intraday_prices(df, interval='1m'):
    symbols = df['Symbol'].tolist()
    intraday_data = []

    for symbol in symbols:
        stock_info = yf.Ticker(symbol)
        today_intraday = stock_info.history(interval=interval, period='1d')
        
        # convert the index to Eastern time zone (eastern is default for NYSE and NASDAQ)
        today_intraday.index = today_intraday.index.tz_convert('US/Eastern')
        
        # find the closest times to 09:33 PST and 07:15 PST
        price_0933 = today_intraday.between_time('09:33', '09:34')['Close']
        price_1015 = today_intraday.between_time('10:15', '10:16')['Close']
        
        # use the first available price in the range if not empty
        price_0933_value = price_0933.values[0] if not price_0933.empty else None
        price_1015_value = price_1015.values[0] if not price_1015.empty else None
        
        intraday_data.append({
            'Symbol': symbol,
            'Price@09:33': price_0933_value,
            'Price@10:15': price_1015_value
        })

    # create a df from intraday data list
    intraday_data_df = pd.DataFrame(intraday_data)
    
    return intraday_data_df

In [145]:
# add stock data to existing df
qstart_bt_df = add_stock_data(premarket_df)

# fetch intraday prices and merge with the existing df
intraday_prices_df = add_intraday_prices(premarket_df)
qstart_bt_df = pd.merge(qstart_bt_df, intraday_prices_df, on='Symbol', how='left')

display(qstart_bt_df)

,Symbol,Current Price,Timestamp,High,Low,Open,Close,Date,Price@09:33,Price@10:15
0,PNW,78.940002,2024-07-11 09:30:00-04:00,80.25,78.93,78.940002,80.010002,2024-07-11 00:00:00-04:00,79.070000,80.144997
1,FRT,104.120003,2024-07-11 09:30:00-04:00,105.559998,103.82,104.230003,104.93,2024-07-11 00:00:00-04:00,NaN,105.220001
2,ROIV,11.410100,2024-07-11 09:30:00-04:00,11.525,11.19,11.45,11.24,2024-07-11 00:00:00-04:00,11.480000,11.445000
3,RBC,285.010010,2024-07-11 09:30:00-04:00,289.51001,285.0,285.01001,287.519989,2024-07-11 00:00:00-04:00,NaN,287.744995
4,NNN,43.270000,2024-07-11 09:30:00-04:00,44.310001,43.23,43.369999,44.110001,2024-07-11 00:00:00-04:00,43.250000,43.910000
5,FLR,45.619999,2024-07-11 09:30:00-04:00,46.790001,45.560001,45.619999,46.549999,2024-07-11 00:00:00-04:00,45.750000,46.160000
6,WBS,45.150002,2024-07-11 09:30:00-04:00,46.639999,45.02,45.150002,46.549999,2024-07-11 00:00:00-04:00,45.320000,45.480000
7,CBSH,57.660000,2024-07-11 09:30:00-04:00,58.189999,57.450001,57.57,57.52,2024-07-11 00:00:00-04:00,57.900002,58.185001
8,RMBS,67.419998,2024-07-11 09:30:00-04:00,67.480003,64.610001,67.419998,65.220001,2024-07-11 00:00:00-04:00,66.904999,66.449997
9,BPOP,91.889999,2024-07-11 09:30:00-04:00,93.160004,91.589996,91.889999,92.959999,2024-07-11 00:00:00-04:00,NaN,92.724998


In [ ]:
#### START HERE SUDZ DEV #############################
# figure out and order your cols
# backtest all of these
# determine default buy, sell, and your exit order simultaneous order
# think about visual chart also
# could have a tear sheat of your record. yes yes yes yes

# you have like 4 dataframes fuck
# simple_gap_df, enhanced_df, premarket_df, and qstart_bt_df

# NEWS also but maybe get to that on the weekend and tee up for next week.

# default buy @ 0632
# default sell @ 0705
# default shares = 1000

In [ ]:
backtest_df_cols = ['Symbol', 
                    'Description', 
                    'Pre-market Change %', 
                    'Pre-market Gap %', 
                    'marketCapType', 
                    'Price', 
                    'Pre-market Open', 
                    'Industry', 
                    'Index', 'Sector', 
                    'Exchange', 
                    'Current Price',
                    'High', 
                    'Low', 
                    'Open', 
                    'Close', 
                    'Date', 
                    'Price@09:33', 
                    'Price@10:15']

In [146]:
# create staging df for high and low prices
high_low_time_df = qstart_bt_df[['Symbol', 'Description', 'High', 'Low']]

print(high_low_time_df)

KeyError: "['Description'] not in index"

| END WORK |
|-|

In [ ]:
# visually verifying is fine
# but we have to put some other reliable mechanism here to create more conviction from screen L1 -> screen L2 -> screen L

In [ ]:
# #### ---- DAY TARGETS ---- ####
# secondary_list = ['ATRA', 'WIX', 'PNW', 'FRT', 'CGNX', 'ROIV', 'RBC',]

# # create day_df from simple gap df applying a filter matching symbols in day_target_list
# second_df = simple_gap_df[simple_gap_df['Symbol'].isin(secondary_list)]

# second_df


| SCREEN LEVEL III |
|-|
| Primary TARGETs (PTs) |
| PTs >= 2 |

In [ ]:
# do i need a time var here for the pre-market prices?
# other apis for pre-market and nasdaq registration
# iex api deprecating in August 2

In [ ]:
# # return primary targets from df
# pt_list = ['ACLS', 'MNSO']
# pt_df = simple_gap_df[simple_gap_df['Symbol'].isin(pt_list)]

# print(pt_list)
# print(pt_df.shape)
# pt_df

['ACLS', 'MNSO']
(0, 15)


,Symbol,Description,Pre-market Change %,Pre-market Gap %,marketCapType,Price,Pre-market Open,Industry,Index,Sector,Exchange,RSI,MA20,MA50,MACD


In [ ]:
#### DEV ####












In [ ]:
#### ---- Quickstart Backtest ---- ####
# manually set target list (>= 2 stocks)
manual_day_list = ['ACLS', 'MNSO',]

print(first_list)
print(secondary_list)
# print(manual_day_list)
print(pt_list)

['WIX', 'PNW', 'FRT', 'CGNX', 'ROIV', 'RBC', 'SFM', 'NNN', 'FLR', 'WBS', 'ENSG', 'CBSH', 'EXP', 'OGE', 'RMBS', 'CMA', 'AGI', 'ZION', 'BPOP', 'SUM', 'JAZZ', 'SNV', 'TMHC', 'VVV', 'WHR', 'MOD', 'SITE', 'COOP', 'R', 'ATKR', 'ZWS', 'FNB', 'KBH', 'AVAV', 'COLB', 'GBCI', 'PAGS', 'FFIN', 'UMBF', 'HGV', 'URBN', 'IPGP', 'ALE', 'SHAK', 'VLY', 'APLE', 'AX', 'QFIN', 'AROC', 'APAM', 'YETI', 'GFF', 'FULT', 'NHI', 'SKT', 'MWA', 'ABR', 'LXP', 'FHB', 'CBU', 'EBC', 'WAFD', 'BFH', 'PPBI']
['ATRA', 'WIX', 'PNW', 'FRT', 'CGNX', 'ROIV', 'RBC']
['ACLS', 'MNSO']


In [ ]:
# return a new dataframe with only the stocks in the manual_day_list
manual_day_df = simple_gap_df[simple_gap_df['Symbol'].isin(manual_day_list)]
# use iloc to select first two cols
manual_day_df = manual_day_df.iloc[:, [0, 1, 8]]

manual_day_df

,Symbol,Description,Index


In [ ]:
# create staging df for high and low prices
high_low_time_df = qstart_bt_df[['Symbol', 'Description', 'High', 'Low']]

print(high_low_time_df)

  Symbol Description                   High        Low        
0  MNSO   MINISO Group Holding Limited   19.549999       18.93
1  ACLS     Axcelis Technologies, Inc.  158.610001  151.669998


In [ ]:
# fetch the timestamps @ high and low price marks
def get_high_low_timestamps(symbol, interval='1m', period='1d'):
    data = yf.Ticker(symbol).history(interval=interval, period=period)
    high_time_stamp = data['High'].idxmax()
    low_time_stamp = data['Low'].idxmin()
    return high_time_stamp, low_time_stamp

# fetch timestamps and update the df
high_low_time_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df['Symbol'].apply(
    lambda symbol: pd.Series(get_high_low_timestamps(symbol))
)

print(high_low_time_df)

  Symbol Description                   High        Low         high_time_stamp           low_time_stamp           
0  MNSO   MINISO Group Holding Limited   19.549999       18.93 2024-07-10 09:48:00-04:00 2024-07-10 15:37:00-04:00
1  ACLS     Axcelis Technologies, Inc.  158.610001  151.669998 2024-07-10 09:45:00-04:00 2024-07-10 09:30:00-04:00


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/118328880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df['Symbol'].apply(
/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/118328880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df['Symbol'].apply(


In [ ]:
# transform time string data
high_low_time_df['high_time_stamp'] = high_low_time_df['high_time_stamp'].dt.time
high_low_time_df['low_time_stamp'] = high_low_time_df['low_time_stamp'].dt.time

high_low_time_df

/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/2157241607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df['high_time_stamp'] = high_low_time_df['high_time_stamp'].dt.time
/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/2157241607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df['low_time_stamp'] = high_low_time_df['low_time_stamp'].dt.time


,Symbol,Description,High,Low,high_time_stamp,low_time_stamp
0,MNSO,MINISO Group Holding Limited,19.549999,18.93,09:48:00,15:37:00
1,ACLS,"Axcelis Technologies, Inc.",158.610001,151.669998,09:45:00,09:30:00


In [ ]:
# add the last two cols to the qstart_bt_df
qstart_bt_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df[['high_time_stamp', 'low_time_stamp']]

# transform Date column string to set the format
qstart_bt_df['Date'] = pd.to_datetime(qstart_bt_df['Date']).dt.strftime('%Y-%m-%d')

print(qstart_bt_df.shape)
qstart_bt_df

,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15,high_time_stamp,low_time_stamp
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10,19.075001,19.229099,09:48:00,15:37:00
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10,153.199997,153.199997,09:45:00,09:30:00


In [ ]:
#### ---- Set Manual Trade Details ---- ####
# structured dictionary for two (2) stocks with buy and sell details
trade_deets = {
    'MNSO': {
        'buy_price': 19.11,
        'buy_time': '09:31:47',
        'quantity': 300,
        'sell_price': 19.40,
        'sell_time': '09:58:19',
        'trading_platform': 'Fidelity'
    },
      'ACLS': {
        'buy_price': 153.20,
        'buy_time': '09:31:10',
        'quantity': 50,
        'sell_price': 158.24,
        'sell_time': '09:55:00',
        'trading_platform': 'Robinhood'
    }
}

In [ ]:
# add trade details to the DataFrame
for symbol in trade_deets:
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'buy_price'] = trade_deets[symbol]['buy_price']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'buy_time'] = trade_deets[symbol]['buy_time']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'quantity'] = trade_deets[symbol]['quantity']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'sell_price'] = trade_deets[symbol]['sell_price']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'sell_time'] = trade_deets[symbol]['sell_time']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'trading_platform'] = trade_deets[symbol]['trading_platform']

print(qstart_bt_df.shape)
qstart_bt_df

(2, 18)


,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15,high_time_stamp,low_time_stamp,buy_price,buy_time,quantity,sell_price,sell_time,trading_platform
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10,19.075001,19.229099,09:48:00,15:37:00,19.11,09:31:47,300.0,19.40,09:58:19,Fidelity
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10,153.199997,153.199997,09:45:00,09:30:00,153.20,09:31:10,50.0,158.24,09:55:00,Robinhood


In [ ]:
# add math calculation columns
qstart_bt_df['amount@buy'] = qstart_bt_df['buy_price'] * qstart_bt_df['quantity']
qstart_bt_df['amount@close'] = qstart_bt_df['sell_price'] * qstart_bt_df['quantity']
qstart_bt_df['realized_$'] = qstart_bt_df['amount@close'] - qstart_bt_df['amount@buy']
qstart_bt_df['realized_%'] = (qstart_bt_df['realized_$'] / qstart_bt_df['amount@buy']) * 100

print(qstart_bt_df.shape)
qstart_bt_df

(2, 22)


,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15,high_time_stamp,low_time_stamp,buy_price,buy_time,quantity,sell_price,sell_time,trading_platform,amount@buy,amount@close,realized_$,realized_%
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10,19.075001,19.229099,09:48:00,15:37:00,19.11,09:31:47,300.0,19.40,09:58:19,Fidelity,5733.0,5820.0,87.0,1.517530
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10,153.199997,153.199997,09:45:00,09:30:00,153.20,09:31:10,50.0,158.24,09:55:00,Robinhood,7660.0,7912.0,252.0,3.289817


In [ ]:
# save day of week as a string
day_of_week = datetime.datetime.today().strftime('%A')
# lowercase and only first 3 chars
day_of_week_abrv = day_of_week[:3]

print(day_of_week)
print(day_of_week_abrv)

Wednesday
Wed


In [ ]:
# create or update existing directory
trade_logz_dir = "tRaDeLoGz"
os.makedirs(trade_logz_dir, exist_ok=True)

# save to xlsx in that directory
trade_logz_dir_file_path = f"{trade_logz_dir}/tRaDeLoGz_{day_of_week_abrv}{today_date_str}.xlsx"
qstart_bt_df.to_excel(trade_logz_dir_file_path, index=False)

In [ ]:
# open the file
os.system(f"open {trade_logz_dir_file_path}")

0

In [ ]:
# START here with the trade details
# later figure out how you can speed things up with news
# keep automating and speeding things up
# reorder the columns
# you can keep improving it every day if you want to, but you will have the trade record avaialable data saved for your records
# next iteration you have to rearrange cols, set sheet number
# also set the col var types properly, i.e., for $ and % and probably others
# should also have some kinda notes.
# maybe even ideal trade?? not sure, isn't that obvious it would be the high and low of the day
# accompany these time stamps with volume somehow
# thinking later of using volume to determine the best time to sell instead of times (which is based on general intraday (early 630-730) volume trends)
# can get fancier with excel output maybe set the zoom, auto open, not sure, could also get as fancy as you want with excel wings
# keep the one file but different excel docouments for each day, for now just one sheet per doc keep simple
# could include type of buy and sell
# should figure out strategy, i.e., some standard type protocols like trailing stop lostt % not sure
# would like to do percenct, maybe use market cap or price to determine bc prob cant be standard for all stocks all time, maybe tho

#### the 2 you really trade. all others you should paper trade and measure, then keep refining the program as needed
# seek advice and outside input

In [ ]:
#### ---- Create Folders ---- ####
# also create excel doc for each trading day, checks for existing folders and files and update or add to them
# reorder cols later
# you did this over in the edgartools, use that to start and copy

In [ ]:
# # save day_df to csv
# day_df.to_csv(f"trading_view_raw_data/tv_screen_gap-up_2024-07-10_612_day_df.csv", index=False)


In [ ]:
#### ---- TRADING NOTES -> DATE = 7/10/24 ---- ####
""" 
MINISO Group Holding Limited (MNSO) - High likelihood to gap up due to strong pre-market activity, positive news, and robust financial performance.
Axcelis Technologies, Inc. (ACLS) - High likelihood to gap up driven by strong industry performance, positive financial results, and favorable technical indicators.
"""

' \nMINISO Group Holding Limited (MNSO) - High likelihood to gap up due to strong pre-market activity, positive news, and robust financial performance.\nAxcelis Technologies, Inc. (ACLS) - High likelihood to gap up driven by strong industry performance, positive financial results, and favorable technical indicators.\n'

In [ ]:
# here is where you are at. 
# aLSO I WANT A is it a good pre-market score before the main screener and then after also accounting for main screener and all pre screeners

In [ ]:
#### you could try in the morning to get IEX news
# maybe a try with the foreign news, IF foreign exchanges = true?
# could also look at the indexes returned in the screener and then chart those after also
# might need to create symbol and name dictionary to work with

# need price
# news
# other markets
# news and translation
# index comparison
# trends

## backtesting 
# openprice, price at 5min, price at 0705, (perc gain), high, low, close
# from here can also indicate shares simulated, etc...

| Backtesting -> START |
|-|
| HEAVY Current DEV |
| < July 4th |

In [ ]:
print(sgap_stock_list)

['MMYT', 'WIX', 'YMM', 'FHN', 'AGNC', 'WAL', 'ETSY', 'ZION', 'STWD', 'ALGM', 'MNSO', 'RITM', 'ACLS', 'ZI', 'COLB', 'AEO', 'PTEN', 'INTR']


In [ ]:
# # List of stock symbols
# symbols = ['SFM', 'ATI', 'AAON', 'MNSO', 'LNTH', 'CVLT', 'CRS', 'SMPL', 'TRMD', 'SDRL', 'UNF', 'GOGL', 'VZIO', 'PAYO']

# # Specify the date and times (in EST)
# date_str = '2024-06-26'  # example date
# # date_str = datetime.now().strftime('%Y-%m-%d')
# times_est = ['09:35', '10:05']  # times in EST

# # Fetch historical data and extract prices at specified times
# data = {'Symbol': [], '$@0935': [], '$@1005': []}

# for symbol in symbols:
#     ticker = yf.Ticker(symbol)
#     # Fetch 1-minute data for the specified date
#     hist = ticker.history(start=date_str, end=(datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1m')
    
#     # Ensure the index is timezone-aware by localizing to UTC if necessary, then convert to EST
#     if hist.index.tz is None:
#         hist.index = hist.index.tz_localize('UTC').tz_convert('US/Eastern')
#     else:
#         hist.index = hist.index.tz_convert('US/Eastern')

#     # Filter data for specified times
#     price_at_0935 = hist.loc[hist.index.strftime('%H:%M') == '09:35']['Close'].values
#     price_at_1035 = hist.loc[hist.index.strftime('%H:%M') == '10:35']['Close'].values
    
#     # Save data
#     data['Symbol'].append(symbol)
#     data['$@0935'].append(price_at_0935[0] if len(price_at_0935) > 0 else None)
#     data['$@1005'].append(price_at_1035[0] if len(price_at_1035) > 0 else None)

# # Convert to DataFrame for better visualization
# df = pd.DataFrame(data)

# print(date_str)
# df

In [ ]:
# symbols = ['SFM', 'ATI', 'AAON', 'MNSO', 'LNTH', 'CVLT', 'CRS', 'SMPL', 'TRMD', 'SDRL', 'UNF', 'GOGL', 'VZIO', 'PAYO']

# # Specify the date (in EST)
# # date_str = '2024-06-26'  # example date
# date_str = datetime.now().strftime('%Y-%m-%d')

# # Fetch historical data and extract Open, Close, High, and Low prices
# data_daily = {'Symbol': [], 'Open': [], 'Close': [], 'High': [], 'Low': []}

# for symbol in symbols:
#     ticker = yf.Ticker(symbol)
    
#     # Fetch daily data for the specified date to get Open, Close, High, and Low prices
#     hist_daily = ticker.history(start=date_str, end=(datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d')
    
#     # Get the daily Open, Close, High, and Low prices
#     open_price = hist_daily['Open'].values
#     close_price = hist_daily['Close'].values
#     high_price = hist_daily['High'].values
#     low_price = hist_daily['Low'].values
    
#     # Save daily data
#     data_daily['Symbol'].append(symbol)
#     data_daily['Open'].append(open_price[0] if len(open_price) > 0 else None)
#     data_daily['Close'].append(close_price[0] if len(close_price) > 0 else None)
#     data_daily['High'].append(high_price[0] if len(high_price) > 0 else None)
#     data_daily['Low'].append(low_price[0] if len(low_price) > 0 else None)

# # Convert to DataFrame for better visualization
# df_daily = pd.DataFrame(data_daily)

# print(date_str)
# df_daily

| Backtesting <- END |
|-|
| HEAVY Current DEV |
| < July 4th |

In [ ]:
# # to csv
# simple_gap_df.to_csv(f"trading_view_raw_data/UPD_{today_date_str}", index=False)

| vectorb Development |
|-|
| START DEV |

In [ ]:
# #### Yahoo Finance API ####
# # yf_vbt_stock = 'NOW' # servicenow
# yf_vbt_stock = sgap_stock_list[0] # first stock in the list
# print(f'${yf_vbt_stock}')
# yahoo_price_data = yf.Ticker(yf_vbt_stock).history(period='1d')

# yahoo_price_close = yahoo_price_data['Close'].iloc[-1]
# yahoo_price_date = yahoo_price_data.index[-1]

# print(f'${yahoo_price_close}')
# print(yahoo_price_date)


In [ ]:
# iex_api_key = os.getenv("IEX_API_KEY")  # Replace with your actual IEX API key

In [ ]:
# # set iex api parameters
# iex_symbol = 'NOW'
# iex_mins_interval = '10'
# # iex_day_range = '10d'

# # iex url construct
# url = f'https://api.iex.cloud/v1/data/core/minutebar/{iex_symbol}?last={iex_mins_interval}&token={iex_api_key}'

# # fetch data
# response = requests.get(url)
# iex_data = response.json()
# print(response.status_code)

# # look at the json
# print(iex_data)
# # # check if data is in expected format and convert to DataFrame
# # if isinstance(iex_data, list):
# #     iex_hd_df = pd.DataFrame(iex_data)
# #     print(iex_hd_df)

In [ ]:
# if isinstance(iex_data, list):
#     parsed_data = []

#     # Parse the JSON data
#     for item in iex_data:
#         for time, data in item.items():
#             if time not in ['id', 'key', 'subkey', 'date', 'updated']:
#                 data['time'] = time
#                 parsed_data.append(data)

#     # Convert to DataFrame
#     iex_hd_df = pd.DataFrame(parsed_data)
#     iex_hd_df['date'] = pd.to_datetime(iex_hd_df['date'], format='%Y%m%d')
#     iex_hd_df['datetime'] = iex_hd_df['date'].astype(str) + ' ' + iex_hd_df['time']
#     iex_hd_df['datetime'] = pd.to_datetime(iex_hd_df['datetime'])
#     iex_hd_df.set_index('datetime', inplace=True)
    
#     # Drop the separate date and time columns
#     iex_hd_df.drop(columns=['date', 'time'], inplace=True)

#     # Display the DataFrame
#     print("DataFrame:")
#     print(iex_hd_df)
# else:
#     print("Unexpected data format:", iex_data)

# # view
# iex_hd_df

In [ ]:
#### historical data
# determine time FRAMEs 
# determine time RANGE
# retrieve historical data from iex cloud api

# what about market volatility trends, times, lunch power, both daily and cyclically, quarterly, event based like earnings, sector etc.

In [ ]:
# # higher frequency
# start_date = '2024-05-24'
# end_date = '2024-05-25'
# vbt_data = vbt.YFData.download('NOW', start=start_date, end=end_date, interval='5m')

In [ ]:
# close_5m = vbt_data.get('Close')

# print(close_5m)

In [ ]:
# # print the day today - 60 days
# dt.date.today() - dt.timedelta(days=60)

In [ ]:
# # symbol = 'NOW'
# symbol = sgap_stock_list[0]
# # set the start date to today -59 days
# start_date = (dt.date.today() - dt.timedelta(days=59)).strftime('%Y-%m-%d')  # Adjust to current date
# end_date = dt.datetime.today().strftime('%Y-%m-%d')  # Adjust to current date

# # download data for each timeframe
# timeframes = ['5m', '15m', '30m', '1h']
# data = {tf: vbt.YFData.download(symbol, start=start_date, end=end_date, interval=tf) for tf in timeframes}

# # calculate SMAs for each timeframe
# fast_sma = {tf: data[tf].get('Close').rolling(window=20).mean() for tf in timeframes}
# slow_sma = {tf: data[tf].get('Close').rolling(window=50).mean() for tf in timeframes}

In [ ]:
# # convert to dataframs
# fast_sma_df = pd.DataFrame(fast_sma)
# slow_sma_df = pd.DataFrame(slow_sma)

In [ ]:
# fast_sma_df.tail(10)

In [ ]:
# print(len(fast_sma_df))
# fast_sma_df.dropna(how='all', inplace=True)
# slow_sma_df.dropna(how='all', inplace=True)
# print(len(fast_sma_df))

| 1-min chart |
|-|

In [ ]:
# first_stock = simple_gap_df['Symbol'].iloc[0]
# print(first_stock)


In [ ]:
# # symbol = 'NOW'
# symbol = simple_gap_df['Symbol'].iloc[0]
# # auto - dev in progress
# start_date = (dt.date.today() - dt.timedelta(days=7)).strftime('%Y-%m-%d')  
# end_date = dt.datetime.today().strftime('%Y-%m-%d')  

# # download data for each timeframe
# timeframes = ['1m']
# one_min_data = {tf: vbt.YFData.download(symbol, start=start_date, end=end_date, interval=tf) for tf in timeframes}

# # calculate SMAs for each timeframe
# one_min_fast_sma = {tf: one_min_data[tf].get('Close').rolling(window=20).mean() for tf in timeframes}
# one_min_slow_sma = {tf: one_min_data[tf].get('Close').rolling(window=50).mean() for tf in timeframes}

In [ ]:
# # convert to dataframs
# one_min_fast_sma_df = pd.DataFrame(one_min_fast_sma)
# one_min_slow_sma_df = pd.DataFrame(one_min_slow_sma)

In [ ]:
# # NaNs check and drop
# print(len(one_min_fast_sma_df))
# one_min_fast_sma_df.dropna(inplace=True)
# one_min_slow_sma_df.dropna(inplace=True)
# print(len(one_min_fast_sma_df))

In [ ]:
# one_min_fast_sma_df.head(4)

In [ ]:
#### merge the dataframes
### add the ticker and description to every row
## could try to get more batches of 7 days
# could also try exploring different apis more

In [ ]:
# # vbt_data = vbt.YFData.download(yf_vbt_stock, start='2024-05-01', end='2024-05-31')
# price = vbt.YFData.download('NOW', start='2024-05-01', end='2024-05-31').get('Close')
# # price = vbt.YFData.download(yf_vbt_stock, start='2024-05-01', end='2024-05-31').get('Close')

# # print(vbt_data)
# print(price)

In [ ]:
#### action on other exchanges

In [ ]:
#### news, foreign exchanges, foreign news, translate.

In [ ]:
#### crew ai setup i feed them the best info possible

# personas
# dev = code review and feedback (ea day), potential model tweaks, could be good ci/cd pipeline servicenow demo
# sector analyst
# market analyst
# resarcher 1
# researcher 2
# human

| vectorb Development| 
|-|
| END DEV|

| guss_pl_df |
|-|
| Gap-up strategy screener, profit & loss dataframe |
| START -> guss |

In [ ]:
#### simulation plan = buy @ 635, sell @ 705 

In [ ]:
# save / update results to excel

| guss_pl_df |
|-|
| END <- guss |

| Indexes |
|-|
| IEX Cloud API $ | 

market indexes, sector indexes
currencies???

In [ ]:
# #### STANDARD MARKET INDEXES EVALUATION
# market_index_list = ['SPY', # SPX = S&P 500 index
#                      'RSP', # SPXEW - S&P 500 equal weight index
#                      'XLRE'] # real estate select sector SPDR fund

# #### IEX CLOUD API KEY -> VARIABLE ####
# iex_api_key = os.getenv("IEX_API_KEY")  # Replace with your actual IEX API key

# # Set the list of symbols variable
# index_symbols = ','.join(market_index_list)

# #### SET DATE RANGE HERE for - historical data processing
# data_range = '5m' 

# url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={index_symbols}&types=chart&range={data_range}&token={iex_api_key}'

# # Make the GET request
# response = requests.get(url)

# # Initialize a DataFrame to collect data for each symbol
# market_index_df = pd.DataFrame()

# # Check if the request was successful
# if response.status_code == 200:
#     # Convert the response to JSON
#     data = response.json()

#     # Process and print the data with a progress bar
#     for symbol in tqdm(market_index_list, desc="Processing symbols"):
#         if symbol in data:
#             # Create a DataFrame for the current symbol's data
#             symbol_df = pd.DataFrame(data[symbol]['chart'])
#             # Add a 'symbol' column to identify the data
#             symbol_df['symbol'] = symbol
#             # Append the current symbol's DataFrame to the main DataFrame
#             market_index_df = pd.concat([market_index_df, symbol_df], ignore_index=True)
# else:
#     print(f'Failed to fetch data: {response.status_code}')

# # Ensure date column is datetime format
# market_index_df['date'] = pd.to_datetime(market_index_df['date'])

# #### FILTER matching MANDATORY columns only
# hist_data_mandatory_cols_list = ['symbol', 'date', 'close', 'high', 'low', 'open', 'volume']
# market_index_df = market_index_df[hist_data_mandatory_cols_list]

# # Sort by date descending and ticker order specified in the list
# market_index_df['symbol'] = pd.Categorical(market_index_df['symbol'], categories=market_index_list, ordered=True)
# market_index_df = market_index_df.sort_values(by=['date', 'symbol'], ascending=[False, True])
# # Reset the index to get a numbered index
# market_index_df = market_index_df.reset_index(drop=True)
# # rename the new index (these are DAYS aka trading periods since today if sorted descending)
# # market_index_df.index.name = 'INDEX'

# # print(market_index_df.columns)

# # # view df
# # market_index_df

In [ ]:
# # Set variables for historical data analysis
# mi_length = len(market_index_df.index)
# unique_stocks = market_index_df['symbol'].nunique()
# start_date = market_index_df['date'].min()
# end_date = market_index_df['date'].max()

# print(f"START date: {start_date}")
# print(f"END date: {end_date}")
# print(f'{market_index_df["date"].nunique()} days of historical stock data.\n')
# print(f"{mi_length} = Length of dataframe")
# print(f"{unique_stocks} = Unique stocks in dataframe")

# # Entries per stock
# days_per_stock = mi_length / unique_stocks
# print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# # View the stocks
# print(f'Unique stock ticker symbols: {market_index_list}')
# index_cols_list = market_index_df.columns.tolist()
# print(f'Columns: {index_cols_list}')

# # convert df to .csv
# # historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# print('INDEX DATAFRAME: historical price data $')
# # Display the first few rows of the dataframe
# market_index_df.tail(10)

| MARKET SENTIMENT | 
|-|

In [ ]:
#### MARKET SENTIMENT
# 1 - VIX (Volatility Index): A high VIX can indicate market uncertainty, making it essential to manage risk carefully.
# 2 - Overall Market Trend: Align your trades with the overall market trend. If major indices are bullish, it supports a positive trading environment.
# 2i - see CRAMER
# 2ii - see CRAMER
# 2iii - see CRAMER
# 3 - maybe some CRAMER + your own adaptation(s)
# 4 - Index match and performance
# 5 - sector index match on performance
# 6 - how do i deal with implied volatility, do i overlay, is there  pre-built metric (if so how is that calculated)
# 7 - further VIX and other adjustments, how to deal with implied volatility, at the macro and specific stock level / sector level and whatever is in between

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [ ]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [ ]:
"""THIS IS WHERE YOU HAVE TO START AND DO WORK, maybe move up to organize better"""

'THIS IS WHERE YOU HAVE TO START AND DO WORK, maybe move up to organize better'

In [ ]:
# """NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
# info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

# info_news_df

In [ ]:
# #### IEX CLOUD API KEY -> VARIABLE ####
# iex_api_key = os.getenv("IEX_API_KEY")

# # set the list of symbols variable
# symbols = ','.join(sgap_stock_list)

# # SETTING THE DATE RANGE
# """
# y - Example: range=1y -> returns (one year)
# ytd - Example: range=ytd -> returns (year-to-date)
# m - Example: range=6m returns (six months)
# d - Example: range=5d returns (five days)
# """
# data_range = '5m' # DATE RANGE (for historical stock price data)

# url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_api_key}'

# # Make the GET request
# response = requests.get(url)

# # Initialize a list to collect DataFrames for each symbol
# dataframes_list = []

# # Check if the request was successful
# if response.status_code == 200:
#     # Convert the response to JSON
#     data = response.json()

#     # Process and print the data with a progress bar
#     for symbol in tqdm(sgap_stock_list, desc="Processing symbols"):
#         if symbol in data:
#             # Create a DataFrame for the current symbol's data
#             symbol_df = pd.DataFrame(data[symbol]['chart'])
#             # Add a 'Ticker' column to identify the data
#             symbol_df['Ticker'] = symbol
#             # Append the current symbol's DataFrame to the list
#             dataframes_list.append(symbol_df)
#             # print(f'Data for {symbol}:')
#             # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
# else:
#     print(f'Failed to fetch data: {response.status_code}')

# # concatenate dataframes and create a combined list
# historical_df = pd.concat(dataframes_list, ignore_index=True)


In [ ]:
# # Set variables for historical data analysis
# historical_length = len(historical_df.index)
# unique_stocks = historical_df['Ticker'].nunique()
# start_date = historical_df['date'].min()
# end_date = historical_df['date'].max()

# print(f"START date: {start_date}")
# print(f"END date: {end_date}")
# print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
# print(f"{historical_length} = Length of dataframe")
# print(f"{unique_stocks} = Unique stocks in dataframe")

# # Entries per stock
# days_per_stock = historical_length / unique_stocks
# print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# # View the stocks
# print(f'Unique stock ticker symbols: {sgap_stock_list}')
# historical_cols_list = historical_df.columns.tolist()
# print(f'Columns: {historical_cols_list}')

# # convert df to .csv
# # historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# # Display the first few rows of the dataframe
# historical_df.tail(10)

In [ ]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
# historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [ ]:
# # Ensure the dataframe is sorted by date if it's not already
# historical_sorted_df = historical_df.sort_values(by='date')

# # Select mandatory columns for a historical price bar chart + volume
# # and explicitly copy the dataframe to ensure it's not a view but a new object
# chart_historical_df = historical_sorted_df[[
#     'symbol',   # *MANDATORY
#     'date',     # *MANDATORY
#     'close',    # *MANDATORY
#     'high',     # *MANDATORY
#     'low',      # *MANDATORY
#     'open',     # *MANDATORY
#     'volume'    # *MANDATORY
# ]].copy()

# # Group by 'symbol' and get the last occurrence
# last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

# last_date_df


In [ ]:
# # re-index the dataframe
# # chart_historical_df.reset_index(drop=True, inplace=True)
# # view head of your main df
# print(f'Length of historical data: {len(chart_historical_df.index)}')

In [ ]:
# you need to get the chart dataframe to a place where it has all the data you need in there. 
# it would be more next level if we let it rip into live stock feeds from here. all setup with the correct lines.
# in the charts live feed we could use the tradingview html widget to display the charts.

In [ ]:
"""END DEV FRIDAY"""

'END DEV FRIDAY'

| KEY LEVELS (Support & Resistance) |
|-|
| TECHNICAL INDICATOR CALCULATIONS FROM HISTORICAL DATA |

in order of priority of in scope. 
thinking of moving some of the levels out as i test them and see what they look like.
for instance i am thinking of eventually completely replacing MAs with EMAs

In [ ]:
# "calculate key levels from historical data"
# # save historical data to a csv
# # chart_historical_df.to_csv(f'chart_historical_gpt_data{today_date_str}.csv', index=False)

# chart_historical_df.head()
# # think about returning unique values first row 
# # you wont get today if b/c of no close price obviously

| HISTORICAL DATA |
|-|
| Baselne Data: OHCLV (Open, High, Low, Close, Volume) Candlestick Chart |
| IMPORTANT: For effective plotting and time series analysis, it's crucial that the date column in the df is in the datetime format rather than an object (string). |

In [ ]:
# chart_historical_df['date'] = pd.to_datetime(chart_historical_df['date'])
# chart_historical_df.set_index('date', inplace=True)

# print(chart_historical_df.index.dtype)  # Should print 'datetime64[ns]'
# print(chart_historical_df.columns)  # This will list all columns except 'date'
# chart_historical_df.head()


# TREND LINES - UNIT TESTING

| TICKER - Unit TESTING | 
|-|

In [ ]:
# #### dynamic symbol test update 
# # return the unique symbols and store as list
# unique_symbols = chart_historical_df['symbol'].unique()
# unique_symbols_list = unique_symbols.tolist()

# print(unique_symbols_list)

# ex_ticker = unique_symbols_list[0]

# print(ex_ticker)

In [ ]:
# # #### HARDCODED SYMBOL -> Option #1
# # ticker = 'CRM'
# # ticker_df = chart_historical_df[chart_historical_df['symbol'] == ticker].copy()

# #### DYNAMIC SYMBOL (allocation) -> Option #2
# ticker = ex_ticker
# ticker_df = chart_historical_df[chart_historical_df['symbol'] == ex_ticker].copy()

# ticker_df.head()

In [ ]:
# # rename columns so they are capitalized for mplfinance specs
# ticker_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

# ticker_df.tail(10)

In [ ]:
# # save unit test dataframe to csv
# ticker_df.to_csv(f'unit_TEST/historical_data_{ticker}{today_date_str}.csv', index=True)

# ADVANCED WIP

In [ ]:
# ticker_df.head(6)

| ADVANCED CHARTING | 
|-|
| Trend Lines | 

In [ ]:
# def check_trend_line(support: bool, pivot: int, slope: float, y: np.array):
#     # compute sum of differences between line and prices, 
#     # return negative value if invalid 
    
#     intercept = -slope * pivot + y.iloc[pivot]  # Using .iloc[] for positional indexing
#     line_vals = slope * np.arange(len(y)) + intercept  # Calculate line values
#     diffs = line_vals - y  # Calculate differences between line values and actual values
    
#     if support and diffs.max() > 1e-5:
#         return -1.0  # Line is not valid for support
#     elif not support and diffs.min() < -1e-5:
#         return -1.0  # Line is not valid for resistance
    
#     err = (diffs ** 2.0).sum()  # Calculate sum of squared errors
#     return err

# def optimize_slope(support: bool, pivot: int, init_slope: float, y: np.array):
#     slope_unit = (y.max() - y.min()) / len(y)  # Determine the unit change for slope
    
#     # Initialize optimization variables
#     opt_step = 1.0
#     min_step = 0.0001
#     curr_step = opt_step
#     best_slope = init_slope
#     best_err = check_trend_line(support, pivot, init_slope, y)
    
#     assert(best_err >= 0.0)  # Initial slope should not produce an error
    
#     get_derivative = True
#     derivative = None
#     while curr_step > min_step:
#         if get_derivative:
#             # Numerical differentiation to find the direction to adjust slope
#             slope_change = best_slope + slope_unit * min_step
#             test_err = check_trend_line(support, pivot, slope_change, y)
#             derivative = test_err - best_err

#             if test_err < 0.0:  # If increasing slope fails, try decreasing
#                 slope_change = best_slope - slope_unit * min_step
#                 test_err = check_trend_line(support, pivot, slope_change, y)
#                 derivative = best_err - test_err

#             if test_err < 0.0:  # If derivative failed, abort optimization
#                 raise Exception("Derivative failed. Check your data.")

#             get_derivative = False

#         if derivative > 0.0:  # If increasing slope increased error
#             test_slope = best_slope - slope_unit * curr_step
#         else:  # If increasing slope decreased error
#             test_slope = best_slope + slope_unit * curr_step
        
#         test_err = check_trend_line(support, pivot, test_slope, y)
#         if test_err < 0 or test_err >= best_err:  # If slope failed or didn't reduce error
#             curr_step *= 0.5  # Reduce step size
#         else:  # If test slope reduced error
#             best_err = test_err
#             best_slope = test_slope
#             get_derivative = True  # Recompute derivative
    
#     return (best_slope, -best_slope * pivot + y.iloc[pivot])  # Using .iloc[] for positional indexing

# def fit_trendlines_single(data: np.array):
#     x = np.arange(len(data))
#     coefs = np.polyfit(x, data, 1)  # Fit line of best fit
#     line_points = coefs[0] * x + coefs[1]  # Get points of line
    
#     # Find pivot points
#     upper_pivot = (data - line_points).argmax()
#     lower_pivot = (data - line_points).argmin()
   
#     # Optimize the slope for both trend lines
#     support_coefs = optimize_slope(True, lower_pivot, coefs[0], data)
#     resist_coefs = optimize_slope(False, upper_pivot, coefs[0], data)

#     return (support_coefs, resist_coefs)

# def fit_trendlines_high_low(high: np.array, low: np.array, close: np.array):
#     x = np.arange(len(close))
#     coefs = np.polyfit(x, close, 1)  # Fit line of best fit
#     line_points = coefs[0] * x + coefs[1]  # Get points of line

#     # Find pivot points
#     upper_pivot = (high - line_points).argmax()
#     lower_pivot = (low - line_points).argmin()
    
#     # Optimize the slope for both trend lines
#     support_coefs = optimize_slope(True, lower_pivot, coefs[0], low)
#     resist_coefs = optimize_slope(False, upper_pivot, coefs[0], high)

#     return (support_coefs, resist_coefs)


In [ ]:
# should be based on 72 periods but displayed as 24 periods maybe? nah?

In [ ]:
# # step 1: fit trendlines for each stock
# support_coefs, resist_coefs = fit_trendlines_high_low(ticker_df['High'], ticker_df['Low'], ticker_df['Close'])

In [ ]:
# # step 2: calculate trendlines
# x_values = np.arange(len(ticker_df))  # Use the actual data range

# # Calculate the line values using the coefficients
# support_line = support_coefs[0] * x_values + support_coefs[1]
# resistance_line = resist_coefs[0] * x_values + resist_coefs[1]

| ADVANCED CHARTING |
|-|
| Line of Best Fit (Regression Line) |
| Pivot point Xs |
| CLOSE |

In [ ]:
# # linear regresion
# last_72_df = ticker_df.iloc[-72:]
# X = np.arange(len(last_72_df)).reshape(-1, 1)
# y = last_72_df['Close'].values.reshape(-1, 1)

# model = LinearRegression().fit(X, y)
# best_fit_last_72 = model.predict(X).flatten()

# # Preparing the regression line plot data
# best_fit_series = pd.Series([np.nan] * len(ticker_df))
# best_fit_series.iloc[-72:] = best_fit_last_72


In [ ]:
# # Identify the pivot points within the last 72 periods
# pivot_point_low_index = last_72_df['Close'].idxmin()
# pivot_point_high_index = last_72_df['Close'].idxmax()

In [ ]:
# # Initialize series for pivot points with NaN values
# pivot_points_high_series = pd.Series(data=[np.nan] * len(ticker_df), index=ticker_df.index)
# pivot_points_low_series = pd.Series(data=[np.nan] * len(ticker_df), index=ticker_df.index)

# # Set the pivot point values
# pivot_points_high_series[pivot_point_high_index] = ticker_df.loc[pivot_point_high_index, 'Close']
# pivot_points_low_series[pivot_point_low_index] = ticker_df.loc[pivot_point_low_index, 'Close']

| ADVANCED CHARTING |
|-|
| Extending the trend and regression lines to the future |

In [ ]:
# # Extend x_values for future predictions
# future_line_periods = 10
# current_periods = len(ticker_df)
# extended_x_values = np.arange(current_periods + future_line_periods)

# # Calculate extended trend lines
# extended_support_line = support_coefs[0] * extended_x_values + support_coefs[1]
# extended_resistance_line = resist_coefs[0] * extended_x_values + resist_coefs[1]

# # Calculate extended line of best fit
# X_future = np.arange(-72, future_line_periods).reshape(-1, 1)  # -72 for last 72 periods and future periods
# extended_best_fit = model.predict(X_future).flatten()

| ADVANCED CHARTING - CHART IT |
|-|
| Candlestick Chart (KEY Levels overlayed) |


In [ ]:
# # prepare the plot
# apds = [mpf.make_addplot(support_line, color='green', width=2),
#         mpf.make_addplot(resistance_line, color='red', width=2),
#         mpf.make_addplot(best_fit_series, color='orange', width=2)]        
#         # mpf.make_addplot(pivot_points_high_series, type='scatter', markersize=100, marker='x', color='green'),
#         # mpf.make_addplot(pivot_points_low_series, type='scatter', markersize=100, marker='x', color='red')]

In [ ]:
# # look at the df
# ticker_df.tail(10)

# # youre going to have to create some placeholders here.
# # another df?

In [ ]:
# #### CHART IT ####
# # Define the period for the x-axis view and future extension
# lookback_periods = 90
# future_periods = 20

# # Calculate the extended index for the lookback period
# lookback_index = ticker_df.index[-lookback_periods]

# # Generate the future index by adding the future periods to the last date
# future_index = pd.date_range(start=ticker_df.index[-1] + pd.Timedelta(days=1),
#                              periods=future_periods, freq='B')

# # Combine the current and future indices
# full_index = ticker_df.index.union(future_index)

# # Set the xlim for plotting
# xlim = (lookback_index, full_index[-1])

# # Plot the candlestick chart with extended x-axis
# mpf.plot(ticker_df,
#          type='candle',
#          style='charles',
#          volume=True,
#          title=f'${ticker} CHART',
#          figratio=(42, 14),
#          xlim=xlim,  # Set the new x-axis limits
#          addplot=apds)  # 'apds' should contain all your additional plots

In [ ]:
# ticker_df.tail(10)

In [ ]:
# support_line

# END ADVANCED WIP TESTS

| DATA PACKAGE |
|-|
| Trend Lines |


In [ ]:
# # Step 3: Calculate the y-values for the line of best fit for the last 72 periods
# best_fit_last_72 = model.predict(X)

# # Create a series for the best fit line, filled with NaN except for the last 72 values
# best_fit_series = pd.Series([np.nan] * len(ticker_df), index=ticker_df.index)
# # best_fit_series = pd.Series(model.predict(X), index=last_72_df.index)
# best_fit_series[-72:] = best_fit_last_72

# # # Step 4: Pivot Points
# # pivot_point_low_index = last_72_df['Close'].idxmin()
# # pivot_point_low_value = last_72_df.loc[pivot_point_low_index, 'Close']

# # pivot_point_high_index = last_72_df['Close'].idxmax()
# # pivot_point_high_value = last_72_df.loc[pivot_point_high_index, 'Close']

# # # Create a series for the pivot points, similar to the best fit line
# # pivot_points_low = pd.Series([np.nan] * len(ticker_df), index=ticker_df.index)
# # pivot_points_low[pivot_point_low_index] = pivot_point_low_value

# # pivot_points_high = pd.Series([np.nan] * len(ticker_df), index=ticker_df.index)
# # pivot_points_high[pivot_point_high_index] = pivot_point_high_value

# # # Step X: chart it
# # last_72_index = ticker_df.index[-90] # 90 periods on the x-axis

# # mpf.plot(ticker_df,
# #          type='candle',
# #          style='charles',
# #          volume=True,
# #          title=f'{ticker} Candlestick Chart: Last 72 Periods with Regression Line and Pivot Point',
# #          figratio=(42, 14),
# #          xlim=(last_72_index, ticker_df.index[-1]),  # Adjusted for the last 72 periods
# #          addplot=apds)

# # mpf.show()


# # # step 5: chart setup
# # apds = [
# #     mpf.make_addplot(best_fit_series, color='orange', width=2),
# #     mpf.make_addplot(pivot_points_low, type='scatter', markersize=200, marker='x', color='red'),
# #     mpf.make_addplot(pivot_points_high, type='scatter', markersize=200, marker='x', color='green')  # Green X for the highest point
# # ]

# # # Step X: chart it
# # last_72_index = ticker_df.index[-90] # 90 periods on the x-axis

# # mpf.plot(ticker_df,
# #          type='candle',
# #          style='charles',
# #          volume=True,
# #          title=f'{ticker} Candlestick Chart: Last 72 Periods with Regression Line and Pivot Point',
# #          figratio=(42, 14),
# #          xlim=(last_72_index, ticker_df.index[-1]),  # Adjusted for the last 72 periods
# #          addplot=apds)

# # mpf.show()

In [ ]:
# # step 5: chart setup
# apds = [
#     mpf.make_addplot(best_fit_series, color='orange', width=2),
#     mpf.make_addplot(pivot_points_low, type='scatter', markersize=200, marker='x', color='red'),
#     mpf.make_addplot(pivot_points_high, type='scatter', markersize=200, marker='x', color='green')  # Green X for the highest point
# ]

In [ ]:
# # Step X: chart it
# last_72_index = ticker_df.index[-90] # 90 periods on the x-axis

# mpf.plot(ticker_df,
#          type='candle',
#          style='charles',
#          volume=True,
#          title=f'{ticker} Candlestick Chart: Last 72 Periods with Regression Line and Pivot Point',
#          figratio=(42, 14),
#          xlim=(last_72_index, ticker_df.index[-1]),  # Adjusted for the last 72 periods
#          addplot=apds)

# mpf.show()


50DSMA - needed bc everyone has it
standard deviations (2 above)
63DMA - makes at least more sense (63 trading days in a quarter)
2dVWAP
3dVWAP
AVWAP

| Dataframe -> .csv file |
|-|

In [ ]:
# # create a csv file with a flat date index (otherwise it'll get lost when saving to csv)
# ticker_csv_df = ticker_df.reset_index()
# # dtype of date
# print(ticker_csv_df['date'].dtype)
# # print index type
# print(ticker_csv_df.index.dtype)

# # save to csv
# ticker_csv_df.to_csv(f'{ticker}_historical_data{today_date_str}', index=False)

# ticker_csv_df.head(2)


In [ ]:
# #### add VWAP to df
# chart_historical_df['VWAP'] = (chart_historical_df['volume'] * (chart_historical_df['high'] + chart_historical_df['low'] + chart_historical_df['close']) / 3).cumsum() / chart_historical_df['volume'].cumsum()

# chart_historical_df.head(4)

In [ ]:
# """SIMPLE MOVING AVERAGES (SMA)"""
# def calculate_sma(dataframe, period):
#     return dataframe['close'].rolling(window=period).mean()

# chart_historical_df['SMA5'] = calculate_sma(chart_historical_df, 5)
# chart_historical_df['SMA10'] = calculate_sma(chart_historical_df, 10)
# chart_historical_df['SMA15'] = calculate_sma(chart_historical_df, 15)

# chart_historical_df.tail(10)


In [ ]:
# """EXPONENTIAL MOVING AVERAGES (EMA)"""
# def calculate_ema(dataframe, period):
#     return dataframe['close'].ewm(span=period, adjust=False).mean()

# # calculate EMAs for the specified periods
# chart_historical_df['EMA5'] = calculate_ema(chart_historical_df, 5)
# chart_historical_df['EMA10'] = calculate_ema(chart_historical_df, 10)
# chart_historical_df['EMA20'] = calculate_ema(chart_historical_df, 20)
# chart_historical_df['EMA50'] = calculate_ema(chart_historical_df, 50)

# chart_historical_df.tail(10)


In [ ]:
# """MOVING AVERAGE CONVERGENCE DIVERGENCE (MACD)"""
# # calculate the MACD line and the Signal line
# chart_historical_df['MACD'] = calculate_ema(chart_historical_df, 12) - calculate_ema(chart_historical_df, 26)
# chart_historical_df['MACD_Signal'] = chart_historical_df['MACD'].ewm(span=9, adjust=False).mean()

# # MACD Histogram -> useful for visualizing momentum and potential buy/sell signals
# chart_historical_df['MACD_Histogram'] = chart_historical_df['MACD'] - chart_historical_df['MACD_Signal']

# chart_historical_df.tail(10)

In [ ]:
# #### DEV - DATA ORGANIZATION ####
# chart_historical_df.columns

In [ ]:
# """BOLLINGER BANDS (BB)"""
# # BB 1 of 2 -> standard bollinger bands (20D CLOSE $)
# period = 20 # SMA periods
# std_dev_multiplier = 2 # standard deviation multiplier

# # line 1: middle band -> simple moving average (SMA)
# chart_historical_df[f'middle_band_SMA{period}'] = chart_historical_df['close'].rolling(window=period).mean()

# # not a line: standard deviation -> used to calculate the upper and lower bands
# chart_historical_df[f'std_dev_SMA{period}'] = chart_historical_df['close'].rolling(window=period).std()

# # lines 2 & 3: upper and lower bands
# chart_historical_df[f'upper_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] + (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)
# chart_historical_df[f'lower_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] - (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)


# # BB 2 of 2 -> typical price bollinger bands (20D TYPICAL $)
# chart_historical_df['typical_price'] = (chart_historical_df['high'] + chart_historical_df['low'] + chart_historical_df['close']) / 3

# # Calculate the SMA of the Typical Price and name it 'middle_band_typical_SMA20'
# chart_historical_df[f'middle_band_typical_SMA{period}'] = chart_historical_df['typical_price'].rolling(window=period).mean()

# # Calculate the standard deviation of the Typical Price
# chart_historical_df[f'std_dev_typical_SMA{period}'] = chart_historical_df['typical_price'].rolling(window=period).std()

# # Calculate the upper and lower bands for the Typical Price and name them accordingly
# chart_historical_df[f'upper_band_typical_SMA{period}'] = chart_historical_df[f'middle_band_typical_SMA{period}'] + (chart_historical_df[f'std_dev_typical_SMA{period}'] * std_dev_multiplier)
# chart_historical_df[f'lower_band_typical_SMA{period}'] = chart_historical_df[f'middle_band_typical_SMA{period}'] - (chart_historical_df[f'std_dev_typical_SMA{period}'] * std_dev_multiplier)

# # view df with new (2) BBs (6 new columns)
# # chart_historical_df.iloc[-1] # last row slice to check the new columns
# chart_historical_df.tail(2)


In [ ]:
# chart_historical_df.head(5)

In [ ]:
# # drop rows that contain NAs (pulled extra data earlier for this purpose)
# chart_historical_df = chart_historical_df.dropna()

# # view the df
# chart_historical_df.head(5)

In [ ]:
# # date column check data type
# chart_historical_df.index

In [ ]:
# # IMPORTANT: reset index before saving to Excel (need this shit flat so we don't lose the 'date' index)
# chart_historical_df.reset_index(inplace=True)

In [ ]:
# chart_historical_df.index

In [ ]:
# chart_historical_df.head(5)

In [ ]:
# print(tv_L2_df.columns)

In [ ]:
# # takes the original DataFrame and a list of stock symbols,
# # returns a dictionary of DataFrames, one for each stock symbol
# def create_symbol_dataframes(df, symbols):
#     symbol_dfs = {}
#     for symbol in symbols:
#         # Filter the DataFrame for the symbol and store it in a dictionary
#         symbol_dfs[f"{symbol}_chart_df"] = df[df['symbol'] == symbol]
    
#     return symbol_dfs


# symbols = mvp_chart_df['symbol'].unique() 
# symbol_dataframes = create_symbol_dataframes(mvp_chart_df, symbols)

# mtn_df = symbol_dataframes['MTN_chart_df']

# print(symbol_dataframes)



| CALCULATIONS - J. Welles Wilder Jr (technical indicators) |
|-|
| *displayed underneath the chart not overlayed* |
| START |

In [ ]:
# """RELATIVE STRENGTH INDEX (RSI)"""
# def calculate_rsi(dataframe, periods=14):
#     close_delta = dataframe['close'].diff()
#     up = close_delta.clip(lower=0)
#     down = -1 * close_delta.clip(upper=0)

#     ma_up = up.rolling(window=periods).mean()
#     ma_down = down.rolling(window=periods).mean()

#     rsi = ma_up / (ma_up + ma_down) * 100
#     return rsi

# # Adding the RSI to your DataFrame
# chart_historical_df['RSI'] = calculate_rsi(chart_historical_df)

# chart_historical_df.tail(10)


In [ ]:
# """AVERAGE TRUE RANGE (ATR)"""
# def calculate_atr(dataframe, period=14):
#     high_low = dataframe['high'] - dataframe['low']
#     high_close = (dataframe['high'] - dataframe['close'].shift()).abs()
#     low_close = (dataframe['low'] - dataframe['close'].shift()).abs()

#     # Calculate the True Range (TR) as the maximum of the three measures
#     tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

#     # Calculate the ATR by taking the rolling mean of the TR
#     atr = tr.rolling(window=period).mean()
    
#     return atr

# # Adding the ATR to your DataFrame
# chart_historical_df['ATR'] = calculate_atr(chart_historical_df)

# chart_historical_df.tail(10)

In [ ]:
# START HERE
# see about if i should be doing a multi-index and group by if it makes the code more efficient and accurate and all that
# simple data plot soon to see what we are working with and if there are bugs

In [ ]:
"""Average Directional Index (ADX)"""

'Average Directional Index (ADX)'

In [ ]:
"""PARABOLIC SAR (PSAR)"""

'PARABOLIC SAR (PSAR)'

| J. Welles Wilder Jr (technical indicators) |
|-|
| END |

In [ ]:
# LINE LEVELS not full calculation dataframe
# part 1 was calculations append
# part 2 is data points append
# part 3 extra - could be plotting past signals (might integrate ai in here like tensorflow)

| CHARTS |
|-|

In [ ]:
# #### xlwings -> dynamic send to excel ####
# # save name for todays returend gap up stocks historical data
# chart_data_file = f'chart_data{today_date_str}.xlsx'

# # reset index before saving to Excel (need
# chart_historical_df.reset_index(inplace=True, drop=True)

# # Save the DataFrame to an Excel file
# chart_historical_df.to_excel(chart_data_file, index=False)

# # Use xlwings to open the file in Excel
# app = xw.App(visible=True)
# book = app.books.open(chart_data_file)

In [ ]:
# # AVAILABLE COLS
# chart_historical_df.columns

In [ ]:
# def plot_symbol_technicals(chart_historical_df, symbol):
#     # FILTER
#     symbol_df = chart_historical_df[chart_historical_df['symbol'] == symbol].copy()
    
#     # print(symbol_df.head(5))
#     # DATA PREP
#     add2plot = [
#         # mpf.make_addplot(symbol_df['SMA5'], color='#7BAFD4', width=1.0, label='SMA5'), 
#         # mpf.make_addplot(symbol_df['SMA10'], color='#003087', width=1.0, label='SMA10'), 
#         mpf.make_addplot(symbol_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP')
#     ]
    
#     # STYLE & PLOT
#     mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})
#     fig, axes = mpf.plot(symbol_df,
#                          type='candle',
#                          addplot=add2plot,
#                          volume=True,
#                          style=mpf_style,
#                         #  style='yahoo',
#                          figratio=(42, 12),
#                          title=f'${symbol} - Historical Data (4 Months)',
#                          tight_layout=True,
#                          returnfig=True)

#     # watermark
#     fig.text(0.5, 0.5, symbol, fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

#     mpf.show()


In [ ]:
# # print a list of the unique stock symbols
# unique_symbols = chart_historical_df['symbol'].unique()
# print(f'Unique stock symbols: {unique_symbols}')

In [ ]:
# #### EXECUTE CHARTING ####
# # DEV
# plot_symbol_technicals(chart_historical_df, 'GRND')

# # # PRO
# # for unique in unique_symbols:
# #     plot_symbol_technicals(chart_historical_df, unique)

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [ ]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [ ]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

In [ ]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [ ]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [ ]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [ ]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [ ]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [ ]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)